In [184]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, math, os
import re
from datetime import datetime
import datetime as dt
from sqlalchemy import create_engine
import pymysql


In [201]:
connect_info = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format('root','benbilly3@','localhost',3306,'InnoproCM')  #1
engine = create_engine(connect_info)

# 爬取團員加入資訊

In [77]:
def crawlLinklist(email,password):    
   #跳過chrome通知允許alert windows
    driverpath="/Users/benbilly3/Desktop/Innopro-master/chromedriver"
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_setting_values' :{'notifications' : 2}}
    options.add_experimental_option('prefs',prefs)

    #操控chromedriver
    browser=webdriver.Chrome(executable_path=driverpath,options = options)#模擬瀏覽器，跳過通知允許
    url='https://www.facebook.com/'
    browser.get(url)
    search=browser.find_element_by_id('email')
    search.send_keys(email)#填mail
#     time.sleep(3)
    search=browser.find_element_by_id('pass')
    search.send_keys(password)#填密碼
    search.send_keys(Keys.RETURN) # hit return after you enter search text,使用Keys物件功能
    browser.get('https://www.facebook.com/groups/1612014199076380/members/')
    #查理王：https://www.facebook.com/groups/1993881810669370/members/
    
    #滾動
    jsCode = "var q=document.documentElement.scrollTop=100000000"
    for i in range(650):
        browser.execute_script(jsCode)
        time.sleep(1)
        
#     isLikesLoaded = False
#     while(isLikesLoaded == False):
#         try:
#             # scroll to btn position
#             seeMoreBtn = browser.find_element_by_xpath(
#                 "//div[@id='groupsMemberSection_recently_joined']")
#             seeMoreBtnOffsetY = seeMoreBtn.location['y'] - 300000
#             browser.execute_script(
#                 "window.scrollTo(0, {});".format(seeMoreBtnOffsetY))
#             seeMoreBtn.click()
#             time.sleep(1)
#         except Exception as e:
#             if NoSuchElementException:
#                 isLikesLoaded = True
#                 print('All likes loaded')
#             else:
#                 print(e)
#                 isLikesLoaded = True
        
    members = []
    likeLis = browser.find_element_by_xpath(
        "//div[@id='groupsMemberSection_recently_joined']").find_elements_by_xpath("//div[@class='clearfix _60rh _gse']")

    for li in likeLis:
        try:
            userATag = li.find_element_by_xpath(".//div[@class='_60ri']")#fbname 

            userATag2 = userATag.find_element_by_xpath(".//a")#fbuid,
            fbuid = userATag2.get_attribute('ajaxify')
            fbuid =fbuid[fbuid.index('&member_id=')+len('&member_id='):fbuid.index('&ref=MEMBER_LIST')]#抽出id
            
            fburl = userATag2.get_attribute('href')#fburl

            joindate = li.find_element_by_xpath(".//div[@class='_60rj']")#joinDate

            userData = {
                'name': userATag.text,
                'fbuid': fbuid,
                'fburl':fburl,
                'joinDate':joindate.text
            }
            members.append(userData)
        except:
            pass
        
    ad=pd.DataFrame(members,index=range(len(members)))
    
    def select_char(str1):#分離出日期
        try:
            res = re.findall(r"\d+\.?\d*", str1)
            date=','.join(res).replace(',','-')
            result=datetime.strptime(date,'%Y-%m-%d')
        except:
            try:
                res= re.findall(r"\d+\.?\d*", str1)
                result=','.join(res).replace(',','-')
                result=int(result)
                result=datetime.now()-dt.timedelta(result*30)#解決'前幾月'顯示加入
                result=datetime.strptime(result.strftime('%Y-%m-%d'), '%Y-%m-%d')#消除秒數
            except:
                result=None
        return result

    ad['joinDate']=ad['joinDate'].apply(lambda s:select_char(s))
    ad=ad[~ad['joinDate'].isna()]#過濾查詢
    ad.to_csv('/Users/benbilly3/Downloads/志同道合團員資訊.csv')
    ad=ad.reset_index()
    ad=ad.iloc[:,1:]
    ad.to_sql(name='TdaGroupMembers', con=engine, if_exists='replace')
    
    return ad

df=crawlLinklist('benbilly3@gmail.com','benbilly3@')
df

[{'name': 'Ben Tsai',
  'fbuid': '100000415902078',
  'fburl': 'https://www.facebook.com/profile.php?id=100000415902078&fref=gm&__tn__=%2Cd-_-R&eid=ARCgBfqIjCsA6W_T07hGtRC461NxADyaO7qgm2XZFKyophhhBSOkl8-b36ODhRN_8zEXvIhGiBAY4EDo&dti=1612014199076380&hc_location=group',
  'joinDate': '海軍－小小兵'},
 {'name': '查李王',
  'fbuid': '100013251333743',
  'fburl': 'https://www.facebook.com/profile.php?id=100013251333743&sk=wall&fref=gm&__tn__=%2Cd-_-R&eid=ARDuXCFvfhUQwbUoTfSglsCM2A9LjS6jpOSzuCCcbAVoE3g27pNR3fCvlzMXltnpr_g0Np2Hn0y9oZ4k&dti=1612014199076380&hc_location=group',
  'joinDate': '台北市'},
 {'name': '徐筱熙',
  'fbuid': '100009066393902',
  'fburl': 'https://www.facebook.com/profile.php?id=100009066393902&sk=wall&fref=gm&__tn__=%2Cd-_-R&eid=ARCMuguFBTJv9VfFRKBc_j7D4Dsve95vIQflnXK3AJF-aw-17H1Uzfr29PlPSO7PIc8kx1PcyeKpama9&dti=1612014199076380&hc_location=group',
  'joinDate': ''},
 {'name': '陳曉丰',
  'fbuid': '100009040713860',
  'fburl': 'https://www.facebook.com/profile.php?id=100009040713860&sk=

In [149]:
# def createSQLDatabase(newDatabaseName):
#     db = pymysql.connect(host='localhost', port=3306, user='root', passwd='benbilly3@', charset='utf8')
#     cursorInsatnce  =db.cursor()  
#     # SQL Statement to create a database
#     sqlStatement = "CREATE DATABASE "+newDatabaseName  
#     # Execute the create database SQL statment through the cursor instance
#     cursorInsatnce.execute(sqlStatement)
# createSQLDatabase('InnoproCM')

# 查理王

In [196]:
def crawlLinklist(email,password):    
   #跳過chrome通知允許alert windows
    driverpath="/Users/benbilly3/Desktop/Innopro-master/chromedriver"
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_setting_values' :{'notifications' : 2}}
    options.add_experimental_option('prefs',prefs)

    #操控chromedriver
    browser=webdriver.Chrome(executable_path=driverpath,options = options)#模擬瀏覽器，跳過通知允許
    url='https://www.facebook.com/'
    browser.get(url)
    search=browser.find_element_by_id('email')
    search.send_keys(email)#填mail
#     time.sleep(3)
    search=browser.find_element_by_id('pass')
    search.send_keys(password)#填密碼
    search.send_keys(Keys.RETURN) # hit return after you enter search text,使用Keys物件功能
    browser.get('https://www.facebook.com/groups/1993881810669370/members/')
    #查理王：https://www.facebook.com/groups/1993881810669370/members/
    
    #滾動
    jsCode = "var q=document.documentElement.scrollTop=100000000"
    for i in range(300):
        browser.execute_script(jsCode)
        time.sleep(1)
        
    members = []
    likeLis = browser.find_element_by_xpath(
        "//div[@id='groupsMemberSection_recently_joined']").find_elements_by_xpath("//div[@class='clearfix _60rh _gse']")

    for li in likeLis:
        try:
            userATag = li.find_element_by_xpath(".//div[@class='_60ri']")#fbname 

            userATag2 = userATag.find_element_by_xpath(".//a")#fbuid,
            fbuid = userATag2.get_attribute('ajaxify')
            fbuid =fbuid[fbuid.index('&member_id=')+len('&member_id='):fbuid.index('&ref=MEMBER_LIST')]#抽出id
            
            fburl = userATag2.get_attribute('href')#fburl

            joindate = li.find_element_by_xpath(".//div[@class='_60rj']")#joinDate

            userData = {
                'name': userATag.text,
                'fbuid': fbuid,
                'fburl':fburl,
                'joinDate':joindate.text
            }
            members.append(userData)
        except:
            pass
        
    ad=pd.DataFrame(members,index=range(len(members)))
    
    def select_char(str1):#分離出日期
        try:
            res = re.findall(r"\d+\.?\d*", str1)
            date=','.join(res).replace(',','-')
            result=datetime.strptime(date,'%Y-%m-%d')
        except:
            try:
                res= re.findall(r"\d+\.?\d*", str1)
                result=','.join(res).replace(',','-')
                result=int(result)
                result=datetime.now()-dt.timedelta(result*30)#解決'前幾月'顯示加入
                result=datetime.strptime(result.strftime('%Y-%m-%d'), '%Y-%m-%d')#消除秒數
            except:
                result=None
        return result

    ad['joinDate']=ad['joinDate'].apply(lambda s:select_char(s))
    ad=ad[~ad['joinDate'].isna()]#過濾查詢
    ad.to_csv('/Users/benbilly3/Downloads/查理王團員資訊.csv')
    ad=ad.reset_index()
    ad=ad.iloc[:,1:]
    ad.to_sql(name='CTGroupMembers', con=engine, if_exists='replace')
    
    return ad



df=crawlLinklist('benbilly3@gmail.com','benbilly3@')
df

,fbuid,fburl,joinDate,name
13,103113957736836,https://www.facebook.com/VDPTaiwan/?ref=gm&__t...,2018-11-13,李王好德
14,100006927070596,https://www.facebook.com/profile.php?id=100006...,2019-09-03,李榮家
15,103113957736836,https://www.facebook.com/VDPTaiwan/?ref=gm&__t...,2019-08-10,李王好德
16,790385240,https://www.facebook.com/andy.chang.9210256?sk...,2019-08-25,張博超
17,100002995154240,https://www.facebook.com/profile.php?id=100002...,2019-08-23,麻糬
18,100000380702074,https://www.facebook.com/paul.chan.1048?sk=wal...,2019-08-19,Paul Chan
19,100001531673526,https://www.facebook.com/lorashi?sk=wall&fref=...,2019-08-18,Chen-yu Shi
20,1614773199,https://www.facebook.com/demonangele?sk=wall&f...,2019-08-18,May Wang
21,567262117,https://www.facebook.com/alantai919?sk=wall&fr...,2019-06-11,Alan Tai
22,1089597305,https://www.facebook.com/profile.php?id=108959...,2019-08-17,楊安宜


In [200]:
df2=df
df2=df2.reset_index()
df2=df2.iloc[:,1:]
df2.to_sql(name='CTGroupMembers', con=engine, if_exists='replace')

In [ ]:
# df2['joinDate']=df2['joinDate'].apply(lambda s:datetime.now()-dt.timedelta(int(df2.iloc[13,-2])*30))